**Imports**

In [1]:
import axelrod as axl

In [27]:
import itertools

import numpy as np

In [3]:
# importing the bayesian code like this for now

from importlib.machinery import SourceFileLoader

bayesian = SourceFileLoader("bayesian", "src/bayesian.py").load_module()

### A few details about `axelrod`

We can use `axelrod` to simulate the interaction of a given memory one strategy and of a sequence.

In [4]:
from axelrod.action import Action

C, D = Action.C, Action.D

Now the variables `C` and `D` correspond to the actions `C` and `D`.

This is how we can define a memory-one player and their initial move.

In [5]:
q = axl.MemoryOnePlayer((1, 1, 1, 1), initial=D)
q

Generic Memory One Player: (1, 1, 1, 1), D

For the sequence we can use the class `axl.Cycler` which takes an input an `str` sequence. If the sequence is smaller than the number of turns the Cycler just keeps cycling over the same sequence.

In [6]:
p = axl.Cycler("CCD")

In [7]:
match = axl.Match(players=[p, q], turns=3)

_ = match.play()

In [8]:
match.result

[(C, D), (C, C), (D, C)]

We can have a match with errors as well:

In [9]:
match = axl.Match(players=[p, q], turns=3, noise=0.1, seed=10)

_ = match.play()

In [10]:
match.result

[(C, C), (C, C), (D, C)]

### Inferring strategies

Let's assume we are trying to fit the 16 pure strategies.

1. Initially we use `axelrod` for getting the co-player's actions.

In [11]:
p = axl.Cycler("CCDCDDDCDC")

In [12]:
match = axl.Match(players=[p, q], turns=10)

_ = match.play()

In [13]:
coplayers_actions = [history[1] for history in match.result]
coplayers_actions

[D, C, C, C, C, C, C, C, C, C]

In [14]:
num_possible_s = 16

In [39]:
last_turn_outcomes = ["open"] + list(itertools.product([C, D], repeat=2))

In [41]:
pure_transitions = list(itertools.product([C, D], repeat=5))

In [42]:
pure_strategies = {f"M{i}": 
               {k:v for k, v in zip(last_turn_outcomes, transitions)} 
               for i, transitions in enumerate(pure_transitions)}

In [43]:
pure_strategies

{'M0': {'open': C, (C, C): C, (C, D): C, (D, C): C, (D, D): C},
 'M1': {'open': C, (C, C): C, (C, D): C, (D, C): C, (D, D): D},
 'M2': {'open': C, (C, C): C, (C, D): C, (D, C): D, (D, D): C},
 'M3': {'open': C, (C, C): C, (C, D): C, (D, C): D, (D, D): D},
 'M4': {'open': C, (C, C): C, (C, D): D, (D, C): C, (D, D): C},
 'M5': {'open': C, (C, C): C, (C, D): D, (D, C): C, (D, D): D},
 'M6': {'open': C, (C, C): C, (C, D): D, (D, C): D, (D, D): C},
 'M7': {'open': C, (C, C): C, (C, D): D, (D, C): D, (D, D): D},
 'M8': {'open': C, (C, C): D, (C, D): C, (D, C): C, (D, D): C},
 'M9': {'open': C, (C, C): D, (C, D): C, (D, C): C, (D, D): D},
 'M10': {'open': C, (C, C): D, (C, D): C, (D, C): D, (D, D): C},
 'M11': {'open': C, (C, C): D, (C, D): C, (D, C): D, (D, D): D},
 'M12': {'open': C, (C, C): D, (C, D): D, (D, C): C, (D, D): C},
 'M13': {'open': C, (C, C): D, (C, D): D, (D, C): C, (D, D): D},
 'M14': {'open': C, (C, C): D, (C, D): D, (D, C): D, (D, D): C},
 'M15': {'open': C, (C, C): D, (C, 

2. We use the `MemoryOne` class we have defined in `bayesian` which gives us the probability of the action $i$ happening after the history $h_t$.

In [47]:
strategies_to_fit = [bayesian.MemoryOne(error=0.001, states_action_dict=value)
                     for value in pure_strategies.values()]

In [48]:
priors = [bayesian.init_prior_uniform(num_possible_s) for _ in range(16)]


# Opening Move

likelihoods = [strategy.likelihood(turn_action,
                                   "open") for strategy in strategies_to_fit]

posteriors = [bayesian.posterior(prior, likelihood, 
                                 priors[:i] + priors[i + 1:],
                        likelihoods[:i] + likelihoods[i + 1:]) for i, (prior, likelihood) 
              in enumerate(zip(priors, likelihoods))]

priors = posteriors 
    
# The rest

for turn, turn_action in enumerate(coplayers_actions[1:]):

    likelihoods = [strategy.likelihood(turn_action,
                                       match.result[turn + 1]) for strategy in strategies_to_fit]

    posteriors = [bayesian.posterior(prior, likelihood, 
                                     priors[:i] + priors[i + 1:],
                            likelihoods[:i] + likelihoods[i + 1:]) for i, (prior, likelihood) 
                  in enumerate(zip(priors, likelihoods))]

    priors = posteriors 

In [52]:
np.array(posteriors).round(3)

16